In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

# Extra Trees

In [5]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [6]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [7]:
tree = ExtraTreesRegressor()
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 91.10 % , error = 12283571587.994453


In [8]:
for i in range (10):
    tree = ExtraTreesRegressor(warm_start = True)
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
    print("{} - Precision = {:.2f} % , error = {}".format(i, precision, error))

0 - Precision = 91.04 % , error = 12364606457.395063
1 - Precision = 91.01 % , error = 12401141721.729986
2 - Precision = 91.01 % , error = 12400642509.67664
3 - Precision = 91.02 % , error = 12382131275.855284
4 - Precision = 91.05 % , error = 12347604610.34062
5 - Precision = 91.03 % , error = 12376616074.84907
6 - Precision = 91.02 % , error = 12395149588.84798
7 - Precision = 91.04 % , error = 12368032690.276468
8 - Precision = 91.03 % , error = 12374965587.690071
9 - Precision = 91.04 % , error = 12367679923.832384


#### Warm start no influye mucho por lo que decidimos no utilizarlo.

## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [4]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [5]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [6]:
lista_criterio = ['mse']
lista_max_depth = [None,1,5,10]
lista_features = [0.4,0.6,1.0]
lista_estimators = [5, 10, 20, 50]

for criterio in lista_criterio:
    for max_depth in lista_max_depth:
        for estimator in lista_estimators:
            for feature in lista_features:

                tree = ExtraTreesRegressor(criterion = criterio, max_depth = max_depth, n_estimators = estimator,\
                                max_features = feature)
                tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply\
                        (lambda x: tree.predict(x)[0],axis = 1)
                precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
                error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                res.append((criterio,max_depth,estimator,feature,precision,error))
                print(criterio,' - ',max_depth,' - ',estimator,' - ',feature)

mse  -  None  -  5  -  0.4
mse  -  None  -  5  -  0.6
mse  -  None  -  5  -  1.0
mse  -  None  -  10  -  0.4
mse  -  None  -  10  -  0.6
mse  -  None  -  10  -  1.0
mse  -  None  -  20  -  0.4
mse  -  None  -  20  -  0.6
mse  -  None  -  20  -  1.0
mse  -  None  -  50  -  0.4
mse  -  None  -  50  -  0.6
mse  -  None  -  50  -  1.0
mse  -  1  -  5  -  0.4
mse  -  1  -  5  -  0.6
mse  -  1  -  5  -  1.0
mse  -  1  -  10  -  0.4
mse  -  1  -  10  -  0.6
mse  -  1  -  10  -  1.0
mse  -  1  -  20  -  0.4
mse  -  1  -  20  -  0.6
mse  -  1  -  20  -  1.0
mse  -  1  -  50  -  0.4
mse  -  1  -  50  -  0.6
mse  -  1  -  50  -  1.0
mse  -  5  -  5  -  0.4
mse  -  5  -  5  -  0.6
mse  -  5  -  5  -  1.0
mse  -  5  -  10  -  0.4
mse  -  5  -  10  -  0.6
mse  -  5  -  10  -  1.0
mse  -  5  -  20  -  0.4
mse  -  5  -  20  -  0.6
mse  -  5  -  20  -  1.0
mse  -  5  -  50  -  0.4
mse  -  5  -  50  -  0.6
mse  -  5  -  50  -  1.0
mse  -  10  -  5  -  0.4
mse  -  10  -  5  -  0.6
mse  -  10  -  5  -  1.

In [7]:
for r in res:
    print("Criterio = {}, max_depth = {}, n_estimators = {}, features = {}, precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3],r[4],r[5]))

Criterio = mse, max_depth = None, n_estimators = 5, features = 0.4, precision = 91.03 % , error = 12368260838.480778
Criterio = mse, max_depth = None, n_estimators = 5, features = 0.6, precision = 91.02 % , error = 12384470260.495207
Criterio = mse, max_depth = None, n_estimators = 5, features = 1.0, precision = 91.03 % , error = 12373499765.923872
Criterio = mse, max_depth = None, n_estimators = 10, features = 0.4, precision = 91.01 % , error = 12402224812.661474
Criterio = mse, max_depth = None, n_estimators = 10, features = 0.6, precision = 91.01 % , error = 12408602399.492172
Criterio = mse, max_depth = None, n_estimators = 10, features = 1.0, precision = 90.98 % , error = 12449835718.443325
Criterio = mse, max_depth = None, n_estimators = 20, features = 0.4, precision = 91.03 % , error = 12372884685.18024
Criterio = mse, max_depth = None, n_estimators = 20, features = 0.6, precision = 91.04 % , error = 12356921945.141111
Criterio = mse, max_depth = None, n_estimators = 20, feature

In [8]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[5] < min_error:
        min_error = r[5]
        tupla_min_error = r
    if r[4] > max_precision:
        max_precision = r[4]
        tupla_max_precision = r
        
print("Mayor precision = Criterio = {}, max_depth = {}, n_estimators = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4],tupla_max_precision[5]))
print("Menor error = Criterio = {}, max_depth = {}, n_estimators = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4],tupla_min_error[5]))

Mayor precision = Criterio = mse, max_depth = None, n_estimators = 50, features = 0.6, precision = 91.07 % , error = 12324003203.223608
Menor error = Criterio = mse, max_depth = None, n_estimators = 50, features = 0.6, precision = 91.07 % , error = 12324003203.223608


### Vemos que los mejores resultados no varian demasiado del default
## Tomamos los parametros por default

In [36]:
#Probamos distintas combinaciones de columnas, esta es la que dio mejores resultados

columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [37]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2017) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [38]:
tree = ExtraTreesRegressor()
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 91.30 % , error = 12005480261.044409


# calculo con los verdaderos datos a analizar

In [39]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [40]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)

In [41]:
resultado = analizar.loc[:,['id','price_usd']]

In [42]:
resultado.to_csv('resultados/Extra_trees.csv', index = False)

# Ahora uso lat y lon en lugar de place name

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),\
                             ['lat', 'lon', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [5]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [6]:
tree = ExtraTreesRegressor()
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 98.48 % , error = 1758674510.4602613


In [7]:
res = []

In [8]:
lista_criterio = ['mse']
lista_max_depth = [None,1,5,10]
lista_features = [0.4,0.6,1.0]
lista_estimators = [5, 10, 20, 50]

for criterio in lista_criterio:
    for max_depth in lista_max_depth:
        for estimator in lista_estimators:
            for feature in lista_features:

                tree = ExtraTreesRegressor(criterion = criterio, max_depth = max_depth, n_estimators = estimator,\
                                max_features = feature)
                tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply\
                        (lambda x: tree.predict(x)[0],axis = 1)
                precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
                error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                res.append((criterio,max_depth,estimator,feature,precision,error))
                print(criterio,' - ',max_depth,' - ',estimator,' - ',feature)

mse  -  None  -  5  -  0.4
mse  -  None  -  5  -  0.6
mse  -  None  -  5  -  1.0
mse  -  None  -  10  -  0.4
mse  -  None  -  10  -  0.6
mse  -  None  -  10  -  1.0
mse  -  None  -  20  -  0.4
mse  -  None  -  20  -  0.6
mse  -  None  -  20  -  1.0
mse  -  None  -  50  -  0.4
mse  -  None  -  50  -  0.6
mse  -  None  -  50  -  1.0
mse  -  1  -  5  -  0.4
mse  -  1  -  5  -  0.6
mse  -  1  -  5  -  1.0
mse  -  1  -  10  -  0.4
mse  -  1  -  10  -  0.6
mse  -  1  -  10  -  1.0
mse  -  1  -  20  -  0.4
mse  -  1  -  20  -  0.6
mse  -  1  -  20  -  1.0
mse  -  1  -  50  -  0.4
mse  -  1  -  50  -  0.6
mse  -  1  -  50  -  1.0
mse  -  5  -  5  -  0.4
mse  -  5  -  5  -  0.6
mse  -  5  -  5  -  1.0
mse  -  5  -  10  -  0.4
mse  -  5  -  10  -  0.6
mse  -  5  -  10  -  1.0
mse  -  5  -  20  -  0.4
mse  -  5  -  20  -  0.6
mse  -  5  -  20  -  1.0
mse  -  5  -  50  -  0.4
mse  -  5  -  50  -  0.6
mse  -  5  -  50  -  1.0
mse  -  10  -  5  -  0.4
mse  -  10  -  5  -  0.6
mse  -  10  -  5  -  1.

In [9]:
for r in res:
    print("Criterio = {}, max_depth = {}, n_estimators = {}, features = {}, precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3],r[4],r[5]))

Criterio = mse, max_depth = None, n_estimators = 5, features = 0.4, precision = 98.35 % , error = 1915206367.0457602
Criterio = mse, max_depth = None, n_estimators = 5, features = 0.6, precision = 98.47 % , error = 1769817440.7403753
Criterio = mse, max_depth = None, n_estimators = 5, features = 1.0, precision = 98.43 % , error = 1820225390.1640992
Criterio = mse, max_depth = None, n_estimators = 10, features = 0.4, precision = 98.42 % , error = 1826549098.0249176
Criterio = mse, max_depth = None, n_estimators = 10, features = 0.6, precision = 98.53 % , error = 1704458798.5872374
Criterio = mse, max_depth = None, n_estimators = 10, features = 1.0, precision = 98.52 % , error = 1711365561.7320929
Criterio = mse, max_depth = None, n_estimators = 20, features = 0.4, precision = 98.54 % , error = 1686884065.0109363
Criterio = mse, max_depth = None, n_estimators = 20, features = 0.6, precision = 98.57 % , error = 1656624489.9679508
Criterio = mse, max_depth = None, n_estimators = 20, featur

In [10]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[5] < min_error:
        min_error = r[5]
        tupla_min_error = r
    if r[4] > max_precision:
        max_precision = r[4]
        tupla_max_precision = r
        
print("Mayor precision = Criterio = {}, max_depth = {}, n_estimators = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4],tupla_max_precision[5]))
print("Menor error = Criterio = {}, max_depth = {}, n_estimators = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4],tupla_min_error[5]))

Mayor precision = Criterio = mse, max_depth = None, n_estimators = 20, features = 0.6, precision = 98.57 % , error = 1656624489.9679508
Menor error = Criterio = mse, max_depth = None, n_estimators = 20, features = 0.6, precision = 98.57 % , error = 1656624489.9679508


## Tomamos los parametros por default, porque no cambia mucho

In [27]:
#Probamos distintas combinaciones de columnas, esta es la que dio mejores resultados

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [28]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [29]:
tree = ExtraTreesRegressor()
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 98.61 % , error = 1605873834.2964687


# Calculo de los verdaderos datos a analizar

In [30]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [31]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)

In [32]:
resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Extra_trees_2.csv', index = False)

# Usando Grid Search y Cross Validation

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [3]:
# Parametros

criterios = ['mse']
max_depth = [None,1,5,10]
features = [0.4,0.6,1.0]
estimators = [5, 10, 20, 30]

parametros = {"criterion" : criterios, "max_depth" : max_depth, \
              "n_estimators" : estimators, "max_features" : features}

iteraciones_cross_validation = 30
arbol = ExtraTreesRegressor()

In [4]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = -1, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 13:48:57 --- 17:10:15 
 Precision: 96.28 
 Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 1.0, 'n_estimators': 10}


In [5]:
#Con 10 iteraciones:    Precision: 93.71       12:32:41 --- 13:34:00 
#               Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 1.0, 'n_estimators': 30}

In [6]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Extra_trees_GridSearch.csv', index = False)

In [9]:
joblib.dump(grid, 'algoritmos/extra_trees.pkl')

['algoritmos/extra_trees.pkl']

## Hago una especie de cross validation con el mejor algoritmo

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [3]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.4)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = ExtraTreesRegressor(criterion = 'mse', max_depth = None, max_features = 1.0, n_estimators = 10)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [4]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Extra_Trees_Cross_Validation.csv', index = False)